<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Data_collection_Photos_for_cnn_dev_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import of the training data

!pip install datasets --quiet

from datasets import load_dataset
from huggingface_hub import login

# First : Create an account on Hugging face, and create also a token

# Connection to hugging face
#login(token="")

dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import numpy as np
import json

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/946k [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/45.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
dev_df = dataset['validation'].to_pandas()
print("Shape :", dev_df.shape)
dev_df.head()

Shape : (300, 7)


,item,name,description,type,category,subcategory,label
0,http://www.wikidata.org/entity/Q15786,1. FC Nürnberg,"German sports club based in Nuremberg, Bavaria",entity,sports,sports club,cultural representative
1,http://www.wikidata.org/entity/Q268530,77 Records,UK record label,entity,music,record label,cultural exclusive
2,http://www.wikidata.org/entity/Q216153,A Bug's Life,1998 animated film directed by John Lasseter a...,entity,comics and anime,animated film,cultural representative
3,http://www.wikidata.org/entity/Q593,A Gang Story,2011 film by Olivier Marchal,entity,films,film,cultural exclusive
4,http://www.wikidata.org/entity/Q192185,Aaron Copland,"American composer, composition teacher, writer...",entity,performing arts,choreographer,cultural representative


In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [ ]:
def get_claims(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims

In [ ]:
def get_sitelinks(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    sitelinks = item.data.get("sitelinks", {})

    return sitelinks

def get_en_wikipedia_url(wikidata_url):

  sitelinks = get_sitelinks(wikidata_url)

  # Find the english wikidepia page
  enwiki = sitelinks.get("enwiki")

  # Return its URL
  return(enwiki.get("url"))

# Test
#wikidata_url = dev_df.iloc[0,0]
#get_en_wikipedia_url(wikidata_url)

In [ ]:
def get_wikipedia_image(wikidata_url):

    # Extractt the english wikipedia page if it exists
    try:
      wikipedia_url = get_en_wikipedia_url(wikidata_url)

    except Exception:
      return None

    # Fighting against the bots protection
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(wikipedia_url, headers=headers)

    # Make sur the response has no error
    if response.status_code == 200:

        # Extract the infobox of the wikipedia page
        soup = BeautifulSoup(response.text, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox'})

        if infobox:
            # Find the image
            img_tag = infobox.find('img')
            if img_tag:
                # Recreate the URL of the image
                img_url = 'https:' + img_tag['src']
                try:
                    img_response = requests.get(img_url, headers=headers)
                    img = Image.open(BytesIO(img_response.content))
                    return np.array(img)
                except:
                    return None
    # If there is an error somewhere, return None
    return None

#Test
#get_wikipedia_image(wikidata_url)

Application to the dev set

In [ ]:
# Application of the extraction of the photos, for the rows 0:500

# Initialization of the new variable
dev_df['image'] = None

# Extraction of the images to the sub dataframe
dev_df['image'] = dev_df['item'].apply(lambda x: get_wikipedia_image(x))

dev_df['image'] = dev_df['image'].apply(lambda x: json.dumps(x.tolist() if x is not None else None))

# Visualisation
dev_df

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q15786,1. FC Nürnberg,"German sports club based in Nuremberg, Bavaria",entity,sports,sports club,cultural representative,"[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0..."
1,http://www.wikidata.org/entity/Q268530,77 Records,UK record label,entity,music,record label,cultural exclusive,null
2,http://www.wikidata.org/entity/Q216153,A Bug's Life,1998 animated film directed by John Lasseter a...,entity,comics and anime,animated film,cultural representative,"[[[65, 128, 58], [62, 125, 55], [67, 126, 58],..."
3,http://www.wikidata.org/entity/Q593,A Gang Story,2011 film by Olivier Marchal,entity,films,film,cultural exclusive,"[[[107, 144, 153], [106, 143, 152], [104, 140,..."
4,http://www.wikidata.org/entity/Q192185,Aaron Copland,"American composer, composition teacher, writer...",entity,performing arts,choreographer,cultural representative,"[[51, 44, 41, 41, 41, 35, 82, 59, 29, 53, 82, ..."
...,...,...,...,...,...,...,...,...
295,http://www.wikidata.org/entity/Q36180,writer,person who uses written words to communicate i...,concept,visual arts,artist,cultural agnostic,"[[[114, 106, 60], [126, 118, 72], [123, 115, 6..."
296,http://www.wikidata.org/entity/Q156316,Würzburg Residence,cultural heritage monument D-6-63-000-454 (0) ...,entity,history,museum,cultural representative,"[[[183, 208, 248], [187, 212, 252], [188, 215,..."
297,http://www.wikidata.org/entity/Q56911,xunfu,high government official of the Chinese Ming a...,concept,politics,politician,cultural exclusive,null
298,http://www.wikidata.org/entity/Q377,Yanka Kupala,Belarusian writer (1882–1942),entity,literature,poet,cultural representative,"[[63, 63, 62, 60, 58, 58, 59, 59, 57, 60, 65, ..."


In [ ]:
dev_df['image'] = dev_df['image'].apply(lambda s: np.array(json.loads(s)) if pd.notnull(s) else None)

Reshape

In [ ]:
def resize_array(image):

  if image is not None:
    image = image.astype(np.uint8)
    img_pil = Image.fromarray(image)
    img_pil = img_pil.convert("RGB")
    img_pil = img_pil.resize((28, 28), Image.Resampling.LANCZOS)

    return np.array(img_pil)

  return None

# Test
#array = dev_df['image'][5]
#print("Before we reshape :", array.shape)
#resized_array = resize_array(array)
#print("After we reshape :", resized_array.shape)

In [ ]:
# Application to the dataset
dev_df['image'] = dev_df['image'].apply(lambda x: resize_array(x))
#dev_df['image'] = dev_df['image'].apply(lambda s: np.array(json.loads(s)) if pd.notnull(s) else None)

In [ ]:
# Exportation

#dev_df.to_csv('dev_df_images.csv', index=False)
dev_df.to_json("dev_df_images.json", orient="records", lines=True)

from google.colab import files
#files.download('dev_df_images.csv')
files.download('dev_df_images.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>